In [1]:
# --- Cell 1: Imports ---
import os
import pandas as pd

In [2]:
#  Dataset setup ---

# Define the root directory of the dataset
dataset_root = r"C:\Users\avira\Documents\Restart_Skills_v2025\GitHub\Technical_Assessment_Junior_ML_Engineer_Biz_Tech_Analytics\Dataset"

# The dataset is split into three folders: train, valid, and test
splits = ["train", "valid", "test"]

In [3]:
# Convert annotations to YOLO format ---

# Iterate through each dataset split (train/valid/test)
for split in splits:
    # Path to the annotation CSV for the current split
    csv_path = os.path.join(dataset_root, split, "_annotations.csv")

    # Load annotations into a DataFrame
    df = pd.read_csv(csv_path)

    # Iterate over each row (annotation) in the DataFrame
    for _, row in df.iterrows():
        # Original image dimensions
        img_w, img_h = row['width'], row['height']

        # Bounding box coordinates in VOC format (xmin, ymin, xmax, ymax)
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        # --- Convert VOC format to YOLO format ---
        # YOLO expects (x_center, y_center, width, height) normalized [0,1]
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        # --- Class mapping ---
        # bare_hand → class 0, gloved_hand → class 1
        class_id = 0 if row['class'] == "bare_hand" else 1

        # --- Save annotations ---
        # Each image gets a corresponding .txt file with YOLO labels
        img_name = os.path.splitext(row['filename'])[0]
        txt_path = os.path.join(dataset_root, split, img_name + ".txt")

        # Append the annotation (one line per bounding box)
        with open(txt_path, "a") as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")
